In [34]:
import cv2
import matplotlib.pyplot as plt
import numpy as np

def extracao(filename):
    Vet = []
    image = cv2.imread(filename)
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    blurred = cv2.GaussianBlur(gray,(5,5),0)
    thresh = cv2.adaptiveThreshold(blurred, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 135, 20)
    thresh = cv2.resize(thresh, dsize = (1520,2280), interpolation=cv2.INTER_AREA)
    
    for i in [6]:#range(15): # na linha 7 da folha esta o erro, primeiro numero já
        for k in range(10):
            
            crop_img = (thresh[10+i*152:(i+1)*152-10, 10+k*152:(k+1)*152-10])
            
            # identifica contornos na imagem
            contours, hierarchy = cv2.findContours(crop_img, cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)
            
            cv2.imshow('img', crop_img)
            cv2.waitKey(0)
            cv2.destroyAllWindows()
            
            area = np.zeros(len(contours))
            for i, ctr in enumerate(contours):
                x, y, w, h = cv2.boundingRect(ctr)
                area[i] = w*h
            bigarea = np.amax(area[0:-1])
            
            for i, ctr in enumerate(contours):
                x, y, w, h = cv2.boundingRect(ctr)
                if bigarea == area[i]:
                    crop_img = crop_img[y:y+h,x:x+w]
            
            cv2.imshow('img', crop_img)
            cv2.waitKey(0)
            cv2.destroyAllWindows()

            m, n = (np.size(crop_img,0),np.size(crop_img,1))
            M = 255*np.ones((int(m+0.5*m), int(m + 0.5*m)))#, dtype = np.int8)
            M[int(0.25*m + 1): int(0.25*m + m) + 1, int((0.25*m + 0.5*m - 0.5*n) + 1): int((0.25*m + 0.5*m - 0.5*n) + n) + 1] = crop_img

            kernel = np.ones((2, 2),np.uint8)
            M = cv2.erode(M, kernel, iterations = 5)
            crop_img = cv2.resize(M,dsize=(28,28),interpolation=cv2.INTER_AREA)
            cv2.imshow('img', crop_img)
            cv2.waitKey(0)
            cv2.destroyAllWindows()
            
            # padrao de entrada para o classificador
            crop_img = np.reshape(crop_img, (784,))
            Vet.append(crop_img)
    return(Vet)

Vet = []
Target = []

for i in [6]:#range(6):
    for k in range(15):
        for j in range(10):
            Target.append(j)
    #Vet = Vet + extracao('folha%i.png'%(i+1))
    Vet = Vet + extracao('folha%i.png'%(i))

np.save('Num.npy',Vet)
np.save('Target.npy',Target)

ValueError: could not broadcast input array from shape (21,40) into shape (40,21)